# Add population data by hectare in the JSON files
Data comes from: https://www.geocat.ch/geonetwork/srv/eng/catalog.search#/metadata/4bfbbf20-d90e-4131-8fe2-4c454ad45c16  

then:    

Download/Geodata (csv) ; tab Geodata ; "Statistik der Bevölkerung und Haushalte (STATPOP), Geodaten 2022" ; "Download publication"

# Dependencies

In [6]:
import os
import json
import pandas as pd
import numpy as np
import json
from pyproj import Transformer

# Load JSON files to be updated

In [4]:
def load_data(path):
    try: 
        with open(path, 'r') as file:
            data = json.load(file)
        return data
    except Exception as e:
        print(f"An error as occured: {e}")
        
def get_file_names(directory):
    return [
        os.path.join(directory, file) for file in os.listdir(directory) if file.endswith('.json')
    ]

In [5]:
directory = r"../data/google_data_isochrone_pop_cgpt"
FILES = get_file_names(directory)
FILES

['../data/google_data_isochrone_pop_cgpt\\Ex1_8004_Zurich_Werdgartengasse_4.json',
 '../data/google_data_isochrone_pop_cgpt\\Ex2_3027_Bern_Colombstrasse_39.json',
 '../data/google_data_isochrone_pop_cgpt\\Ex3_1006_Lausanne_Av_d_Ouchy_58.json',
 '../data/google_data_isochrone_pop_cgpt\\Ex4_8355_Aadorf_Bruggwiesenstrasse_5.json',
 '../data/google_data_isochrone_pop_cgpt\\Ex5_6319_Allenwinden_Winzruti_39.json',
 '../data/google_data_isochrone_pop_cgpt\\Ex6_8005_Zurich_Heinrichstrasse_200.json',
 '../data/google_data_isochrone_pop_cgpt\\Ex7_8003_Zurich_Birmensdorferstrasse_108.json']

# Function to delete pop data (needed in case radius has to be adapted)

In [35]:
def deletes_population_data_of_existing_JSONfile(path):

    # loads the json file
    with open(path, 'r') as file:
        data = json.load(file)
    if 'population' not in data.keys():
        print(path +' : \nThis JSON file does not have has population data.')
        return
    else:
        # deletes the population data 
        del data['population'] 
        # overwrites the file:
        with open(path, 'w') as file:
            json.dump(data, file, indent=2) # indent = 2 since in the highest level

In [36]:
for file in FILES:
    deletes_population_data_of_existing_JSONfile(file)


# Function to add pop data

In [32]:
def add_population_data_to_existing_JSONfile(path, pop, Radius = 900):
    """Adds population count PER NEIGHBORHOOD to an existing json file.
    As well a radius used and also all the population data (the heactare squares used and their corresponding data).
    Does not do anything if the file already contains the population data.
    Arg:
    - path : path incl. extension ".json"
    - pop : csv file contaning the relevant population data
    - Radius: should correspond to the radius used to generate the JSON file. Rule of thumb, 10min walking = 900m
    """
    # loads the json file
    with open(path, 'r') as file:
        data = json.load(file)
    if 'population' in data.keys():
        print(path +' : \nThis JSON file already has population data.')
        return
    else:
        # get coordinates of the original address:
        loc = data['original_address']['coordinates']
        # transform these coords into the Swiss system LV95:
        transfo_coords = Transformer.from_crs('EPSG:4326','EPSG:2056')
        loc_LV95 = transfo_coords.transform(loc[0],loc[1])

        # get center of each 100x100m square:
        pop['E_KOORD_center'] = pop['E_KOORD']+50
        pop['N_KOORD_center'] = pop['N_KOORD']+50

        # first subsetting: a square of "a bit more than Radius" meters on each side of the property address:
        North_max = loc_LV95[1] + Radius*1.25
        North_min = loc_LV95[1] - Radius*1.25
        East_max = loc_LV95[0] + Radius*1.25
        East_min = loc_LV95[0] - Radius*1.25

        pop_neighborhood = pop[ (pop['N_KOORD_center'] > North_min) & 
                                (pop['N_KOORD_center'] < North_max) &
                                (pop['E_KOORD_center'] > East_min) &
                                (pop['E_KOORD_center'] < East_max)    ]

        # second subsetting: from this "a bit more than" Radius*Radius square, compute distances between each hectare-center and the property address. And Keep only those < Radius meters.
        # define function:
        def distance_to_origin(E_orig, N_orig, E_target, N_target):
            dist = np.sqrt( (E_orig -  E_target)**2 + (N_orig - N_target)**2  )
            return dist
        # apply on dataframe:
        pop_neighborhood['dist'] = pop_neighborhood.apply(lambda row: distance_to_origin(loc_LV95[0], loc_LV95[1], row['E_KOORD_center'], row['N_KOORD_center']), axis = 1)
        # filter:
        pop_neighborhood = pop_neighborhood[pop_neighborhood['dist'] < Radius]
        # get total population in the neighborhood:
        total_pop = int(pop_neighborhood['B22BTOT'].sum()) # float is apparently not supported in a dict.
        # gets the coordinates (center of the squares) in case we want to plot the squares:
        # define function:
        def lv95_to_WGS84(E,N):
            E_wgs84, N_wgs84 = Transformer.from_crs('EPSG:2056','EPSG:4326').transform(E,N)
            return  E_wgs84, N_wgs84
        # apply on dataframe:
        pop_neighborhood[['E_wgs84', 'N_wgs84']] = pop_neighborhood.apply(lambda row: lv95_to_WGS84(row['E_KOORD_center'],row['N_KOORD_center'] ) , axis = 1, result_type='expand')
        # convert relevant columns to a dict:
        STATPOP_squares = pop_neighborhood[['B22BTOT','E_wgs84', 'N_wgs84']].to_dict(orient='records')

        # appends the population data to the existing dictionary:
        data['population'] = dict({'total_pop':total_pop, 'Radius_meters': Radius , 'STATPOP_squares': STATPOP_squares })
        # overwrites the file:
        with open(path, 'w') as file:
            json.dump(data, file, indent=2) # indent = 2 since in the highest level
    

In [37]:
# loads the population data file, only the necessary columns (E_KOORD, N_KOORD, B22TOT)
pop = pd.read_csv('../data/PopDataHectare2022/STATPOP2022.csv', sep= ";", usecols=['E_KOORD', 'N_KOORD', 'B22BTOT'])

for file in FILES:
    add_population_data_to_existing_JSONfile(file, pop, Radius = 750)

C:\Users\philo\AppData\Local\Temp\ipykernel_13680\2266401570.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pop_neighborhood['dist'] = pop_neighborhood.apply(lambda row: distance_to_origin(loc_LV95[0], loc_LV95[1], row['E_KOORD_center'], row['N_KOORD_center']), axis = 1)
C:\Users\philo\AppData\Local\Temp\ipykernel_13680\2266401570.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pop_neighborhood['dist'] = pop_neighborhood.apply(lambda row: distance_to_origin(loc_LV95[0], loc_LV95[1], row['E_KOORD_

# old trials

## load pop data

In [10]:
pop = pd.read_csv('../data/PopDataHectare2022/STATPOP2022.csv', sep= ";")

In [11]:
pop.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 346994 entries, 0 to 346993
Data columns (total 80 columns):
 #   Column    Non-Null Count   Dtype
---  ------    --------------   -----
 0   RELI      346994 non-null  int64
 1   E_KOORD   346994 non-null  int64
 2   N_KOORD   346994 non-null  int64
 3   B22BTOT   346994 non-null  int64
 4   B22B11    346994 non-null  int64
 5   B22B12    346994 non-null  int64
 6   B22B13    346994 non-null  int64
 7   B22B14    346994 non-null  int64
 8   B22B15    346994 non-null  int64
 9   B22B16    346994 non-null  int64
 10  B22B21    346994 non-null  int64
 11  B22B22    346994 non-null  int64
 12  B22B23    346994 non-null  int64
 13  B22B24    346994 non-null  int64
 14  B22B25    346994 non-null  int64
 15  B22B26    346994 non-null  int64
 16  B22B27    346994 non-null  int64
 17  B22B28    346994 non-null  int64
 18  B22B29    346994 non-null  int64
 19  B22B30    346994 non-null  int64
 20  B22BMTOT  346994 non-null  int64
 21  B22BM01   

according to file "..\data\raw\PopDataHectare2022\be-b-00.03-10-STATPOP-v122-tab.xlsx" the useful data (total permanent residents in 2022 - latest data) is in column "B22TOT"; which is the 4th column. => drop all columns after this one.  

E and N KOORD are displayed in the SwissSystem (LV95), where 1 unit = 1 meter. The coordinates given here correspond to the South East corner of the hectare square.


## get rid of useless data

In [13]:
pop = pd.read_csv('../data/PopDataHectare2022/STATPOP2022.csv', sep= ";", usecols=[0,1,2,3])
pop

,RELI,E_KOORD,N_KOORD,B22BTOT
0,48621114,2486200,1111400,6
1,48621115,2486200,1111500,3
2,48631114,2486300,1111400,4
3,48631115,2486300,1111500,3
4,48631117,2486300,1111700,3
...,...,...,...,...
346989,83101690,2831000,1169000,5
346990,83101692,2831000,1169200,3
346991,83111692,2831100,1169200,3
346992,83111693,2831100,1169300,13


In [14]:
# positon ourselves in the middle of each square:
pop['E_KOORD_center'] = pop['E_KOORD']+50
pop['N_KOORD_center'] = pop['N_KOORD']+50

In [15]:
pop.head()

,RELI,E_KOORD,N_KOORD,B22BTOT,E_KOORD_center,N_KOORD_center
0,48621114,2486200,1111400,6,2486250,1111450
1,48621115,2486200,1111500,3,2486250,1111550
2,48631114,2486300,1111400,4,2486350,1111450
3,48631115,2486300,1111500,3,2486350,1111550
4,48631117,2486300,1111700,3,2486350,1111750


## get a sample data : Heinrichstrasse 200

In [7]:
import json
def load_data(path):
    try: 
        with open(path, 'r') as file:
            data = json.load(file)
        return data
    except Exception as e:
        print(f"An error as occured: {e}")

In [17]:
path = FILES[5] # Heinrichstrasse 200
ad = load_data(path)
loc = ad['original_address']['coordinates']
loc

[47.3877722, 8.5254298]

## convert coordinates into LV95 system to select the closest hectares - squares:

In [18]:
from pyproj import Transformer

transfo_coords = Transformer.from_crs('EPSG:4326','EPSG:2056')
loc_LV95 = transfo_coords.transform(loc[0],loc[1])
loc_LV95
# (2682058.511398227, 1249117.163971719)
# check if correct on: https://map.geo.admin.ch/#/map?lang=de&center=2682045.76,1249156.16&z=11.507&bgLayer=ch.swisstopo.pixelkarte-farbe&topic=ech&swisssearch=2682058.511398227,+1249117.163971719&layers=ch.bav.haltestellen-oev,f;ch.swisstopo.swisstlm3d-wanderwege,f,1;ch.swisstopo.hangneigung-ueber_30;ch.swisstopo-karto.skitouren;ch.bafu.wrz-wildruhezonen_portal;ch.bafu.wrz-jagdbanngebiete_select
# ... YES ! 

(2682058.511398227, 1249117.163971719)

In [19]:
transfo_coords

<Concatenated Operation Transformer: pipeline>
Description: Inverse of CH1903+ to WGS 84 (1) + Swiss Oblique Mercator 1995
Area of Use:
- name: Liechtenstein; Switzerland.
- bounds: (5.96, 45.82, 10.49, 47.81)

## find all squares which are ca. 900m away from this location loc_LV95

In [20]:
Radius_1 = 1000
North_max = loc_LV95[1] + Radius_1
North_min = loc_LV95[1] - Radius_1
East_max = loc_LV95[0] + Radius_1
East_min = loc_LV95[0] - Radius_1

pop_neighborhood = pop[ (pop['N_KOORD_center']>North_min) & 
                        (pop['N_KOORD_center']<North_max) &
                        (pop['E_KOORD_center']>East_min) &
                        (pop['E_KOORD_center']<East_max)  ]

print(East_max, North_max) # Milchbuck
print(East_max, North_min) # Zürich HB
print(East_min, North_max) # Fischerweg/Limmatalstrasse
print(East_min, North_min) # BullingerHof

pop_neighborhood
# 237 rows! too many of course, since I have here a square and would need a circle...

2683058.511398227 1250117.163971719
2683058.511398227 1248117.163971719
2681058.511398227 1250117.163971719
2681058.511398227 1248117.163971719


,RELI,E_KOORD,N_KOORD,B22BTOT,E_KOORD_center,N_KOORD_center
228295,68112481,2681100,1248100,16,2681150,1248150
228296,68112482,2681100,1248200,104,2681150,1248250
228297,68112483,2681100,1248300,119,2681150,1248350
228298,68112484,2681100,1248400,310,2681150,1248450
228299,68112485,2681100,1248500,219,2681150,1248550
...,...,...,...,...,...,...
232326,68302496,2683000,1249600,169,2683050,1249650
232327,68302497,2683000,1249700,162,2683050,1249750
232328,68302498,2683000,1249800,73,2683050,1249850
232329,68302499,2683000,1249900,60,2683050,1249950


## compute distance from each square to the original address

In [21]:
import numpy as np

def distance_to_origin(E_orig, N_orig, E_target, N_target):
    dist = np.sqrt( (E_orig -  E_target)**2 + (N_orig - N_target)**2  )
    return dist

In [22]:
pop_neighborhood['dist'] = pop_neighborhood.apply(lambda row: distance_to_origin(loc_LV95[0], loc_LV95[1], row['E_KOORD_center'], row['N_KOORD_center']), axis = 1)

C:\Users\philo\AppData\Local\Temp\ipykernel_13680\2633492607.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pop_neighborhood['dist'] = pop_neighborhood.apply(lambda row: distance_to_origin(loc_LV95[0], loc_LV95[1], row['E_KOORD_center'], row['N_KOORD_center']), axis = 1)


### and filter it

In [23]:
pop_neighborhood = pop_neighborhood[pop_neighborhood['dist']<900]
pop_neighborhood

,RELI,E_KOORD,N_KOORD,B22BTOT,E_KOORD_center,N_KOORD_center,dist
228472,68122489,2681200,1248900,167,2681250,1248950,825.611576
228473,68122490,2681200,1249000,362,2681250,1249050,811.296296
228474,68122491,2681200,1249100,56,2681250,1249150,809.177907
228642,68132489,2681300,1248900,123,2681350,1248950,727.964419
228643,68132490,2681300,1249000,290,2681350,1249050,711.687713
...,...,...,...,...,...,...,...
231863,68282492,2682800,1249200,155,2682850,1249250,802.558171
231864,68282493,2682800,1249300,152,2682850,1249350,825.025347
231865,68282494,2682800,1249400,249,2682850,1249450,858.623333
232101,68292490,2682900,1249000,175,2682950,1249050,894.015059


## transform the coordinates of the squares which are selected to WGS84 

In [24]:
def lv95_to_WGS84(E,N):
    E_wgs84, N_wgs84 = Transformer.from_crs('EPSG:2056','EPSG:4326').transform(E,N)
    return  E_wgs84, N_wgs84

pop_neighborhood[['E_wgs84', 'N_wgs84']] = pop_neighborhood.apply(lambda row: lv95_to_WGS84(row['E_KOORD_center'],row['N_KOORD_center'] ) , axis = 1, result_type='expand')
pop_neighborhood

C:\Users\philo\AppData\Local\Temp\ipykernel_13680\687907453.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pop_neighborhood[['E_wgs84', 'N_wgs84']] = pop_neighborhood.apply(lambda row: lv95_to_WGS84(row['E_KOORD_center'],row['N_KOORD_center'] ) , axis = 1, result_type='expand')
C:\Users\philo\AppData\Local\Temp\ipykernel_13680\687907453.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pop_neighborhood[['E_wgs84', 'N_wgs84']] = pop_neighborhood.apply(lambda row: lv95_to_WGS84(row['E_KOORD_center'],ro

,RELI,E_KOORD,N_KOORD,B22BTOT,E_KOORD_center,N_KOORD_center,dist,E_wgs84,N_wgs84
228472,68122489,2681200,1248900,167,2681250,1248950,825.611576,47.386369,8.514693
228473,68122490,2681200,1249000,362,2681250,1249050,811.296296,47.387268,8.514711
228474,68122491,2681200,1249100,56,2681250,1249150,809.177907,47.388168,8.514729
228642,68132489,2681300,1248900,123,2681350,1248950,727.964419,47.386357,8.516017
228643,68132490,2681300,1249000,290,2681350,1249050,711.687713,47.387256,8.516035
...,...,...,...,...,...,...,...,...,...
231863,68282492,2682800,1249200,155,2682850,1249250,802.558171,47.388868,8.535936
231864,68282493,2682800,1249300,152,2682850,1249350,825.025347,47.389767,8.535954
231865,68282494,2682800,1249400,249,2682850,1249450,858.623333,47.390666,8.535973
232101,68292490,2682900,1249000,175,2682950,1249050,894.015059,47.387057,8.537223


## make a dict out of these squares to export them to JSON

In [25]:
STATPOP_squares = pop_neighborhood[['B22BTOT','E_wgs84', 'N_wgs84']].to_dict(orient='records')
STATPOP_squares[:5]

[{'B22BTOT': 167, 'E_wgs84': 47.38636905471943, 'N_wgs84': 8.514692659241192},
 {'B22BTOT': 362, 'E_wgs84': 47.38726839234171, 'N_wgs84': 8.51471081919575},
 {'B22BTOT': 56, 'E_wgs84': 47.38816772971132, 'N_wgs84': 8.514728979765653},
 {'B22BTOT': 123, 'E_wgs84': 47.38635671413268, 'N_wgs84': 8.516016883906},
 {'B22BTOT': 290, 'E_wgs84': 47.38725605154865, 'N_wgs84': 8.516035066221654}]